In [1]:
import os
import json
import re
import pandas as pd
import numpy as np

In [5]:
DIRNAME = os.path.abspath('')
THETIS_CSV_PATH = os.path.join(DIRNAME, "../files_computed/thetis_all_with_computed.csv")
WWW_DATA_SHIP_TYPES_PATH = os.path.join(DIRNAME, "../../www/views/doc/ship_types.11tydata.json")

In [6]:
df_thetis = pd.read_csv(THETIS_CSV_PATH).replace({np.nan: None})

In [7]:
ship_types = df_thetis["ship_type"].value_counts()
ship_types

Bulk carrier                  7257
Oil tanker                    3750
Container ship                3491
Chemical tanker               2658
General cargo ship            2310
Vehicle carrier                857
Ro-pax ship                    723
Gas carrier                    659
Ro-ro ship                     523
LNG carrier                    449
Passenger ship                 306
Refrigerated cargo carrier     276
Other ship types               242
Container/ro-ro cargo ship     139
Combination carrier             21
Name: ship_type, dtype: int64

In [11]:
with open(WWW_DATA_SHIP_TYPES_PATH, 'w') as f:
    json.dump({ "shipTypesValueCounts": ship_types.to_dict() }, f, indent=2)
print(f"wrote file to {WWW_DATA_SHIP_TYPES_PATH}")

wrote file to /Users/adipasquale/dev/greenferries/data/notebooks/../../www/views/doc/ship_types.11tydata.json


In [13]:
df_thetis[df_thetis.ship_type == "Ro-pax ship"]

,Unnamed: 0,imo,name,ship_type,reporting_period,technical_efficiency,port_of_registry,home_port,ice_class,doc_issue_date,...,voluntary_reporting_average_co2_emissions_per_transport_work_on_laden_voyages_pax,voluntary_reporting_average_co2_emissions_per_transport_work_on_laden_voyages_freight,voluntary_reporting_additional_comments,voluntary_reporting_average_density_of_the_cargo_transported,annual_computed_distance,annual_computed_distance_km,annual_computed_pax,annual_computed_freight,annual_computed_average_speed,annual_computed_ratio_co2_from_pax
6,6,7043843,TALOS,Ro-ro ship,2018,EEDI (60.41 gCO₂/t·nm),Piraeus,Piraeus,None,25/07/2019,...,None,None,None,None,49224.2,91163.3,None,None,23.7701,None
15,15,7226952,FJARDVAGEN,Ro-ro ship,2018,EEDI (313.8 gCO₂/t·nm),None,None,IA,21/05/2019,...,None,None,None,None,40288.2,74613.9,None,None,24.8862,None
39,39,7529964,GEROITE NA ODESSA,Ro-ro ship,2018,EIV (32.5 gCO₂/t·nm),Varna,None,IB,31/05/2019,...,None,None,None,None,15822.5,29303.4,None,None,18.2667,None
40,40,7529976,GEROITE NA SEVASTOPOL,Ro-ro ship,2018,EIV (32.5 gCO₂/t·nm),Varna,None,IB,31/05/2019,...,None,None,None,None,11477.4,21256.2,None,None,16.5744,None
53,53,7802067,FESTIVO,Ro-ro ship,2018,EIV (18.99 gCO₂/t·nm),Las Palmas de Gran canaria,Malaga,IA,19/06/2019,...,None,None,None,None,47262.9,87531,None,None,18.165,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23447,23447,9831165,SIXTINE,Ro-ro ship,2019,EEDI (9.48 gCO₂/t·nm),Valletta,None,None,01/05/2020,...,None,None,None,None,32339.9,59893.5,None,None,26.3813,None
23448,23448,9831177,HERMINE,Ro-ro ship,2019,EEDI (7.28 gCO₂/t·nm),Valletta,None,None,21/07/2020,...,None,None,None,None,33252.1,61582.9,None,None,26.0847,None
23470,23470,9832585,HOLLANDIA SEAWAYS,Ro-ro ship,2019,EEDI (9.34 gCO₂/t·nm),Copenhagen,None,None,27/04/2020,...,None,None,None,None,7737.08,14329.1,None,None,31.8565,None
23587,23587,9848467,ALF POLLAK,Ro-ro ship,2019,EEDI (12.3 gCO₂/t·nm),Cagliari,None,None,30/04/2020,...,None,None,None,None,121683,225356,None,None,34.1563,None
